# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

In [1]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self.remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text


    def remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "user":
                new_statement = {"role": "USER", "message": statement["content"]}
            elif statement["role"] == "llm":
                new_statement = {"role": "CHATBOT", "message": statement["content"]}
            elif statement["role"] == "preamble":
                new_statement = {"role": "SYSTEM", "message": statement["content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

In [2]:
from redisvl.extensions.session_manager import SemanticSessionManager
user_session = SemanticSessionManager(name='llm_chef', session_tag='123', user_tag='abc')
user_session.set_preamble("You are a helpful chef, assisting people in making delicious meals")

client = CohereClient()

10:00:07 redisvl.index.index INFO   Index already exists, not overwriting.


On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [3]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Sure! Here are a few breakfast ideas:

- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.
- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to create a smooth and creamy smoothie. Pour it into a bowl and top it with sliced fruit, nuts, seeds, or granola for a hearty and nutritious breakfast.
- Oatmeal: Oatmeal is a warm and filling breakfast option that can be prepared in a variety of ways. You can cook oats with milk or water and top them with fresh or dried fruit, nuts, seeds, cinnamon, or honey. You can also make overnight oats by mixing oats with yogurt and fruit and letting them soak in the refrigerator overnight.
- Eggs: Eggs are a classic breakfast option and can be prepare

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [4]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Sure, here's a basic recipe for fluffy pancakes:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 cups milk
- 2 eggs
- 3 tablespoons butter, melted
- 1/2 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.
2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla (if using).
3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter, as this can make the pancakes tough. It's okay if there are a few lumps in the batter.
4. Heat a nonstick pan or griddle over medium heat and grease it with butter or cooking spray.
5. Pour 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface of the pancakes, about 2-3 minutes.
6. Flip the pancakes and cook for an additional 1-2 minutes o

In [5]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to create a smooth and creamy smoothie. Pour it into a bowl and top it with sliced fruit, nuts, seeds, or granola for a hearty and nutritious breakfast.\n- Oatmeal: Oatmeal is a warm and filling breakfast option that can be prepared in a variety of ways. You can cook oats with milk or water and top them with fresh or dried fruit, nuts, seeds, cinnamon, or honey. You can also make overnight 

In [6]:
prompt = "I am vegetarian. Can you remove the eggs?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! Here's an egg-free version of the fluffy pancakes recipe:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 cups milk (dairy or non-dairy alternative)
- 1/4 cup vegetable oil or melted butter
- 1/2 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.
2. In a separate bowl, whisk together the milk, vegetable oil or melted butter, and vanilla (if using).
3. Pour the wet ingredients into the dry ingredients and stir until just combined. Again, be careful not to overmix the batter. A few lumps are okay.
4. Heat a nonstick pan or griddle over medium heat and grease it with butter or cooking spray.
5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, about 2-3 minutes.
6. Flip the pancakes carefully and cook for an additional 1-2 minute

In [7]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to create a smooth and creamy smoothie. Pour it into a bowl and top it with sliced fruit, nuts, seeds, or granola for a hearty and nutritious breakfast.\n- Oatmeal: Oatmeal is a warm and filling breakfast option that can be prepared in a variety of ways. You can cook oats with milk or water and top them with fresh or dried fruit, nuts, seeds, cinnamon, or honey. You can also make overnight 

In [8]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Sure, to make the recipe completely vegan, you can replace the butter with a vegan alternative. Here's the updated recipe:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 cups plant-based milk (such as soy, almond, or oat milk)
- 1/4 cup vegetable oil or melted vegan butter
- 1/2 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.
2. In a separate bowl, whisk together the plant-based milk, vegetable oil or melted vegan butter, and vanilla (if using).
3. Pour the wet ingredients into the dry ingredients and gently mix until just combined. It's important not to overmix the batter to ensure fluffy pancakes.
4. Heat a nonstick pan or griddle over medium heat. You can grease it with a little bit of vegan butter or a neutral-flavored oil.
5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until you

In [9]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to create a smooth and creamy smoothie. Pour it into a bowl and top it with sliced fruit, nuts, seeds, or granola for a hearty and nutritious breakfast.\n- Oatmeal: Oatmeal is a warm and filling breakfast option that can be prepared in a variety of ways. You can cook oats with milk or water and top them with fresh or dried fruit, nuts, seeds, cinnamon, or honey. You can also make overnight 

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that can break quickly.

In [10]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.get_recent(top_k=3)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure, here's the first recipe for fluffy pancakes that I provided:

Ingredients:

- 2 cups all-purpose flour
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 2 cups milk
- 2 eggs
- 3 tablespoons butter, melted
- 1/2 teaspoon vanilla extract (optional)

Instructions:

1. In a large bowl, whisk together the flour, baking powder, and salt.
2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla (if using).
3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter, as it should still have some lumps.
4. Heat a nonstick pan or griddle over medium heat and grease it with butter or cooking spray.
5. Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, which should take about 2-3 minutes.
6. Flip the pancakes and continue cooking for an additional 1-2 m

In [11]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me the recipe for those pancakes?'}
{'role': '_llm', '_content': "Sure, here's a basic recipe for fluffy pancakes:\n\nIngredients:\n\n- 2 cups all-purpose flour\n- 2 teaspoons baking powder\n- 1/2 teaspoon salt\n- 2 cups milk\n- 2 eggs\n- 3 tablespoons butter, melted\n- 1/2 teaspoon vanilla extract (optional)\n\nInstructions:\n\n1. In a large bowl, whisk together the flour, baking powder, and salt.\n2. In a separate bowl, whisk together the milk, eggs, melted butter, and vanilla (if using).\n3. Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter, as this can make the pancakes tough. It's okay if there are a few lumps in the batter.\n4. Heat a nonstick pan or griddle over medium heat and grease it with butter or cooking spray.\n5. Pour 1/4 cup of batter onto the pan for each pa

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_relevant()` method.

In [12]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.get_relevant(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Sure! Here's a simple recipe for avocado toast:

Ingredients:
- 1 avocado
- 1 slice of bread (any type you prefer)
- Lemon juice (optional)
- Salt and pepper to taste
- Sliced tomato or a fried egg (optional)

Instructions:
1. Toast your bread to your desired level of doneness.
2. While the bread is toasting, slice the avocado in half and remove the pit. Scoop out the avocado flesh into a small bowl.
3. Using a fork, smash the avocado until it reaches your desired level of chunkiness. You can leave it fairly smooth or keep it more textured.
4. Add a squeeze of lemon juice (if using) to the avocado and season with salt and pepper to taste. Mix gently to combine.
5. Spread the avocado mixture onto the toasted bread.
6. If desired, top with sliced tomato or a fried egg.
7. Enjoy your delicious and nutritious avocado toast!

Feel free to get creative and add your own twist to this basic recipe. You can try adding different seasonings or toppin

In [13]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to create a smooth and creamy smoothie. Pour it into a bowl and top it with sliced fruit, nuts, seeds, or granola for a hearty and nutritious breakfast.\n- Oatmeal: Oatmeal is a warm and filling breakfast option that can be prepared in a variety of ways. You can cook oats with milk or water and top them with fresh or dried fruit, nuts, seeds, cinnamon, or honey. You can also make overnight 

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.

In [14]:
user_session.clear()